In [3]:
#this notebook is to try and implement waifu images from scratch
import torch
import torch.nn as nn
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.animation as animation
from IPython.display import HTML

Doing this training for spellbrush, goal is to recreate anime faces first using pytorch, then using JAX as well and finally try to use diffusion instead of GAN.

<img src="https://i.imgur.com/6NMdO9u.png">

There are two neural networks: a Generator and a Discriminator. The generator generates a "fake" sample given a random vector/matrix, and the discriminator attempts to detect whether a given sample is "real" (picked from the training data) or "fake" (generated by the generator). Training happens in tandem: we train the discriminator for a few epochs, then train the generator for a few epochs, and repeat. This way both the generator and the discriminator get better at doing their jobs.


<img src="https://i.imgur.com/NaKtJs0.png">
Anime Face Dataset
, which consists of over 63,000 cropped anime faces:https://www.kaggle.com/splcher/starter-anime-face-dataset


In [27]:
# setting basic setiing
dataroot = "~moutazelias/Desktop/Desktop/Github/Datasets/animefacedataset/"          # Root directory for dataset
Image_Size =64              # Number of workers for dataloader
batch_size = 128            # Batch size during training
lr = 0.0002                 # Learning rate for optimizers
num_epochs=100              # Number of training epochs
ngpu = 0                    # Number of GPUs available. Use 0 for CPU mode.


In [29]:
# Create the dataset
dataset = ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(Image_Size),
                               transforms.CenterCrop(Image_Size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,shuffle=True)

In [30]:
# Decide which device we want to run on (if i am using google colab or not)
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [33]:
def show_images(images):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid(images.detach(), nrow=8).permute(1, 2, 0))

def show_batch(dl):
    for images, _ in dl:
        show_images(images)
        break
